# 信息矩阵

考虑某个状态$\xi$, 以及一次与该变量相关的观测$\textbf{r}_i$。由于噪声的存在, 观测服从概率分布$p(\textbf{r}_i|\xi)$。  
多次观测时, 各个测量值相互独立, 则多个测量$\textbf{r} = (\textbf{r}_1,\dots,\textbf{r}_n)^T$构建的似然概率为:
$$p(\textbf{r}|\xi) = \Pi_i p(\textbf{r}_i|\xi) \tag{1}$$
如果知道机器人状态的先验信息$p(\xi)$, 如GPS、车轮码盘信息等, 则根据 Bayes 法则, 有后验概率:
<span id='eq.2'></span>
$$p(\xi|\textbf{r}) = \frac{p(\textbf{r}|\xi)p(\xi)}{p(\textbf{r})} \tag{2}$$
通过最大后验估计, 获得系统状态的最优估计:
<span id='eq.3'></span>
$$\xi_\text{MAP} = \arg \min\limits_{\xi} p(\xi|\textbf{r}) \tag{3}$$


后验公式([2](#eq.2))中分母跟状态量无关, 舍弃。最大后验变成了:
$$\xi_\text{MAP} = \arg \min\limits_{\xi} \Pi p(\textbf{r}_i|\xi)p(\xi) \tag{4}$$
即
$$\xi_\text{MAP} = \arg \min\limits_{\xi} \left[ -\sum_i \log p(\textbf{r}_i|\xi) - \log p(\xi) \right] \tag{5}$$
如果假设观测值服从多元高斯分布:
$$ p(\textbf{r}_i|\xi) = \mathscr{N}\left(\mu_i, \sum\nolimits_i \right),\ p(\xi) = \mathscr{N}\left(\mu_{\xi}, \sum\nolimits_{\xi} \right) \tag {6}$$
则有:
$$\xi_\text{MAP} = \arg \min\limits_{\xi} \sum_i \left\| \textbf{r}_i - \mu_i \right\|_{\sum_i}^2 + \left\| \xi - \mu_{\xi} \right\|_{\sum_{\xi}}^2 \tag{7} $$

## case 1
设$x_2$为室外的温度, $x_1$, $x_3$分别为房间1和房间3的室内温度:
$$\begin{align} x_2 & = v_2 \\
x_1 & = w_1 x_2 + v_1 \\
x_3 & = w_3 x_2 + v_3 \tag{10}\end{align}$$
其中, $v_i$ 相互独立, 且各自服从零均值, 协方差为$\sigma_i^2$的高斯分布。

从上述关系, 根据协方差公式的计算方式, 我们可以写出$x$的协方差矩阵, 先从对角元素开始计算:
$$\sum_{11} = E(x_1 x_1) = E\left((\color{red}{w_1} \color{green}{v_2} + \color{red}{v_1})(\color{red}{w_1} \color{green}{v_2} + \color{red}{v_1})\right) = \color{red}{w_1^2} E(\color{green}{v_2^2}) + 2\color{red}{w_1}E(\color{red}{v_1}\color{green}{v_2}) + E(\color{red}{v_1^2}) = \color{red}{w_1^2} \color{green}{\sigma_2^2} + \color{red}{\sigma_1^2} \tag{11}$$
同理有$\sum_{22} = \color{green}{\sigma_2^2}$, $\sum_{33} = \color{blue}{w_3^2} \color{green}{\sigma_2^2} + \color{blue}{\sigma_3^2}$。对于协方差矩阵的非对角元素:
$$\begin{align} \sum_{12} & = E(x_1 x_2) = E\left((\color{red}{w_1} \color{green}{v_2} + \color{red}{v_1}) \color{green}{v_2}\right) = \color{red}{w_1} \color{green}{\sigma_2^2} \\ 
\sum_{13} & = E\left((\color{red}{w_1} \color{green}{v_2} + \color{red}{v_1})(\color{blue}{w_3} \color{green}{v_2} + \color{blue}{v_3})\right) = \color{red}{w_1}\color{blue}{w_3}\color{green}{\sigma_2^2} 
\end{align} \tag{12}$$
以此类推, 可以得到整个**协方差矩阵**:
$$ \sum = \left[ \begin{matrix}
\color{red}{w_1^2} \color{green}{\sigma_2^2} + \color{red}{\sigma_1^2} & \color{red}{w_1}\color{green}{\sigma_2^2} & \color{red}{w_1} \color{blue}{w_3} \color{green}{\sigma_2^2} \\
\color{red}{w_1} \color{green}{\sigma_2^2} & \color{green}{\sigma_2^2} & \color{blue}{w_3} \color{green}{\sigma_2^2} \\
\color{red}{w_1} \color{blue}{w_3} \color{green}{\sigma_2^2} & \color{blue}{w_3} \color{green}{\sigma_2^2} & \color{blue}{w_3^2} \color{green}{\sigma_2^2} + \color{blue}{\sigma_3^2}
\end{matrix} \right] \tag{13} $$

通过计算联合高斯分布从而得到协方差矩阵的逆:
$$\begin{align} p(x1, x2, x3) & = p(x2)p(x1|x2)p(x3|x2) \\
& = \color{green}{\frac{1}{Z_2} \exp \left( -\frac{x_2^2}{2\sigma_2^2} \right)} \color{red}{\frac{1}{Z_1} \exp \left( -\frac{(x_1 - w_1 x_2)^2}{2\sigma_1^2} \right)} \color{blue}{\frac{1}{Z_3} \exp \left(-\frac{(x_3 - w_3 x_2)^2}{2\sigma_3^2} \right)} \end{align} \tag{14}$$

利用指数性质求出联合概率分布:
$$\begin{align} p(x1, x2, x3) & = \frac{1}{Z} \exp \left(\color{green}{-\frac{x_2^2}{2\sigma_2^2}} \color{red}{-\frac{(x_1 - w_1 x_2)^2}{2\sigma_1^2}} \color{blue}{-\frac{(x_3 - w_3 x_2)^2}{2\sigma_3^2}} \right) \\
& = \frac{1}{Z} \exp \left(-x_2^2 \left[\color{green}{\frac{1}{2\sigma_2^2}} \color{red}{+\frac{w_1^2}{2\sigma_1^2}} \color{blue}{+\frac{w_3^2}{2\sigma_3^2}} \right] - x_1^2 \color{red}{\frac{1}{2\sigma_1^2}} + 2 x_1 x_2 \color{red}{\frac{w_1}{2\sigma_1^2}} - x_3^2 \color{blue}{\frac{1}{2\sigma_3^2}} + 2 x_3 x_2 \color{blue}{\frac{w_3}{2\sigma_3^2}} \right) \\
& = \exp \left( -\frac{1}{2} \left[ \begin{matrix} x_1 & x_2 & x_3 \end{matrix} \right]
\left[ \begin{matrix}
\color{red}{\frac{1}{\sigma_1^2}} & \color{red}{-\frac{w_1}{\sigma_1^2}} & 0 \\
\color{red}{-\frac{w_1}{\sigma_1^2}} & \color{red}{\frac{w_1^2}{\sigma_1^2}} \color{green}{+\frac{1}{\sigma_2^2}} \color{blue}{+\frac{w_3^2}{\sigma_3^2}} & \color{blue}{-\frac{w_3}{\sigma_3^2}} \\
0 & \color{blue}{-\frac{w_3}{\sigma_3^2}} & \color{blue}{\frac{1}{\sigma_3^2}}
\end{matrix} \right] 
\left[ \begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix} \right] \right)
\end{align} \tag{15}$$

由此得到**协方差矩阵的逆**, 即**信息矩阵**:
$$\Lambda = \sum^{-1} = \left[ \begin{matrix}
\color{red}{\frac{1}{\sigma_1^2}} & \color{red}{-\frac{w_1}{\sigma_1^2}} & 0 \\
\color{red}{-\frac{w_1}{\sigma_1^2}} & \color{red}{\frac{w_1^2}{\sigma_1^2}} \color{green}{+\frac{1}{\sigma_2^2}} \color{blue}{+\frac{w_3^2}{\sigma_3^2}} & \color{blue}{-\frac{w_3}{\sigma_3^2}} \\
0 & \color{blue}{-\frac{w_3}{\sigma_3^2}} & \color{blue}{\frac{1}{\sigma_3^2}}
\end{matrix} \right] \tag{16}$$
注意: 信息矩阵中有两个元素为0, 它有什么具体含义呢?协方差逆矩阵中如果坐标为$(i, j)$的元素为0, 表示元素$i$和$j$关于其他变量条件独立, 上面的例子中意味着变量$x_1$和$x_3$关于$x_2$条件独立。


- 协方差中非对角元素$\sum_{ij} > 0$表示两变量是正相关。
- 信息矩阵中非对角元素为负数, 甚至为0。$\Lambda_{12} < 0$表示在变量$x_3$发生的条件下, 元素$x_1$和$x_2$正相关。

## case 2
$$\begin{align}
x_2 & = w_1 x_1 + w_3 x_3 + v_2 \tag{17} \\
x_1 &= v_1 \\
x_3 &= v_3 \\
\end{align}$$
协方差矩阵:
$$ \sum = \left[ \begin{matrix}
\color{red}{\sigma_1^2} & \color{red}{w_1}\color{red}{\sigma_1^2} & 0 \\
\color{red}{w_1}\color{red}{\sigma_1^2} & \color{green}{\sigma_2^2} \color{red}{+w_1^2 \sigma_1^2} \color{blue}{+w_3^2 \sigma_3^2} & \color{blue}{w_3} \color{blue}{\sigma_3^2} \\
0 & \color{blue}{w_3} \color{blue}{\sigma_3^2} & \color{blue}{\sigma_3^2}
\end{matrix} \right]$$
协方差矩阵中非对角元素为0表示变量之间没有相关性。这是否意味着信息矩阵中也会为0呢？

